# Newton's Method

#### Taylor Expansion
Estimation of a function value at a given point, coming from a starting point with known function value. E.g. we are at x = (2, 3)' and know f(2, 3) = 20, we wonder what f(3, 4) is.

##### First-Order Taylor Expansion (repetition from last year's Statistical Inference lecture, chapter 2)
For linear functions (planes), an exact determination of a function value at a given point is possible with a first-order Taylor expansion

In [14]:
import numpy as np

In [15]:
x_start = np.array([2, 3])

In [16]:
# Some linear function: 5 + 3x1 + 3x2 + ...
def lin_f(x):
    function_value = 5 + np.sum(3 * x) # slope is 3 in all dimensions
    return function_value

# example output
lin_f(x_start)

20

In [4]:
# Gradient: (3, 3, ...)'
def dlin_f(x):
    return x * 0 + 3 # x * 0 is only to get a 0 vector of same dimension as x

# example output
dlin_f(x_start)

array([3, 3])

In [5]:
# Taylor expansion (first-order)
x_sought = np.array([3, 4])

def taylor1(x_start, f, df, x_sought):
    f_sought_est = f(x_start) + df(x_start) @ (x_sought - x_start) # @: matrix product
    return f_sought_est

#check
estimation = taylor1(x_start, lin_f, dlin_f, x_sought)
print(estimation)
print(lin_f(x_sought))


26
26


We could exactly "estimate" lin_f(3, 5) with a first-order Taylor expansion.

##### Second-Order Taylor Expansion (this is used in the book)
For quadratic functions, an exact determination of a function value at a given point is possible with a second-order Taylor expansion

In [6]:
# Some quadratic function: 5 + 3x1^2 + 3x2^2 + ...
def quad_f(x):
    function_value = 5 + np.sum(3 * x**2)
    return function_value

# example output
quad_f(x_start)

44

In [7]:
# Gradient: (6x1, 6x2, ...)'
def dquad_f(x):
    return x * 6

# example output
dquad_f(x_start)

array([12, 18])

In [8]:
# Hessian:
def ddquad_f(x):
    identity_mat = np.eye(len(x), dtype=float)
    return identity_mat * 6

# example output
ddquad_f(x_start)

array([[6., 0.],
       [0., 6.]])

In [9]:
# Taylor expansion (second-order)
x_sought = np.array([3, 4])

def taylor2(x_start, f, df, ddf, x_sought):
    delta = (x_sought - x_start)
    f_sought_est = f(x_start) + df(x_start) @ delta + 0.5 * delta @ ddquad_f(x_start) @ delta 
    return f_sought_est

#check
estimation = taylor2(x_start, quad_f, dquad_f, ddquad_f, x_sought)
print("second-order Taylor estimation: " + str(estimation))
print("real value: quad_f(x_new) = : " + str(quad_f(x_sought)))

#estimate quadratic function with a first-order taylor expansion
print("first-order Taylor estimation: " + str(taylor1(x_start, quad_f, dquad_f, x_sought)))


second-order Taylor estimation: 80.0
real value: quad_f(x_new) = : 80
first-order Taylor estimation: 74


We could exactly "estimate" quad_f(3, 5) = 80 with a second-order Taylor expansion. The result of the first-order Taylor expansion is just an approximation.

-> for higher-order polynomials or non-plynomial-functions the Taylor estimation can be off, especially when delta is big, e.g. if the function value of a point is estimated that is far away from the current/starting point.

#### Newton Algorithm
The Newton algorithm seeks the root, we want an x_sought where f(x_sought) = 0. We can set the Taylor estimation of f(x_sought) to zero and then analytically determine the x_sought point where f(x_sought) is estimated to be zero according to the Taylor expansion. 

If the target function is a polynomial and the Taylor expansion has at least the order of that polynomial, the Newton Algorithm finds the exact root with one step. But for most target functions (non-plynomial or polynomial of higher-order), the point where the root was estimated to be turns out to not (exactly) be the root of the function. The estimated x_sought is now seen as the new staring point x_start and another step of the newton algorithm is started. We do as many steps until the estimations for the root don't change much between steps.

In [10]:
# Example on the given quadratic function
# Determination of the exact root of a second-order Taylor series (as used in the Goodfellow book)
# step 1 (all we need for a quadratic function):
x_sought = x_start - np.linalg.inv(ddquad_f(x_start)) @ dquad_f(x_start)
x_sought

array([0., 0.])

In [11]:
quad_f(np.array([0, 0]))

5

The second-order Newton algorithm only needed one step for the best possible result for the given quadratic function. 5 is not 0, so (0, 0)' is not a root of quad_f, but it is the lowest possible point due to the intercept of 5. The full second-order Newton algorithm that allows for multiple steps is as follows:

In [12]:
def newton2(x_start, df, ddf, stop_delta = 0.1):
    # Stopping criterion: stop when the last two estimated points are close to each other.
    # E.g. when the Euclidian distance between the last two estimated points (estimation_delta) 
    # is smaller or equal to a defined stop_delta
    
    # Initialization, just to get the while loop running
    estimation_delta = stop_delta + 1
    
    while estimation_delta > stop_delta: 
        # a step
        print("step")
        
        x_sought = x_start - np.linalg.inv(ddf(x_start)) @ df(x_start)
        
        # result comparison with previous step
        estimation_delta = np.sqrt(x_start @ x_sought) # used Euclidian distance here
        
        # x_sought becomes the x_start of the next step:
        x_start = x_sought
    
    return x_sought

In [13]:
newton2(np.array([5, 3]), dquad_f, ddquad_f)

step


array([0., 0.])

After only 1 step, the second-order Newton algorithm reached the global minimum of the quadratic function quad_f().